# 自动微分
:label:`sec_autograd`

正如 :numref:`sec_calculus`中所说，求导是几乎所有深度学习优化算法的关键步骤。
虽然求导的计算很简单，只需要一些基本的微积分。
但对于复杂的模型，手工进行更新是一件很痛苦的事情（而且经常容易出错）。

深度学习框架通过自动计算导数，即*自动微分*（automatic differentiation）来加快求导。
实际中，根据设计好的模型，系统会构建一个*计算图*（computational graph），
来跟踪计算是哪些数据通过哪些操作组合起来产生输出。
自动微分使系统能够随后反向传播梯度。
这里，*反向传播*（backpropagate）意味着跟踪整个计算图，填充关于每个参数的偏导数。

## 一个简单的例子

作为一个演示例子，(**假设我们想对函数$y=2\mathbf{x}^{\top}\mathbf{x}$关于列向量$\mathbf{x}$求导**)。
首先，我们创建变量`x`并为其分配一个初始值。


In [ ]:
import torch

x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

[**在我们计算$y$关于$\mathbf{x}$的梯度之前，需要一个地方来存储梯度。**]
重要的是，我们不会在每次对一个参数求导时都分配新的内存。
因为我们经常会成千上万次地更新相同的参数，每次都分配新的内存可能很快就会将内存耗尽。
注意，一个标量函数关于向量$\mathbf{x}$的梯度是向量，并且与$\mathbf{x}$具有相同的形状。


In [ ]:
x.requires_grad_(True)  # 等价于x=torch.arange(4.0,requires_grad=True)
x.grad  # 默认值是None

(**现在计算$y$。**)


In [ ]:
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

`x`是一个长度为4的向量，计算`x`和`x`的点积，得到了我们赋值给`y`的标量输出。
接下来，[**通过调用反向传播函数来自动计算`y`关于`x`每个分量的梯度**]，并打印这些梯度。


In [ ]:
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

函数$y=2\mathbf{x}^{\top}\mathbf{x}$关于$\mathbf{x}$的梯度应为$4\mathbf{x}$。
让我们快速验证这个梯度是否计算正确。


In [ ]:
x.grad == 4 * x

tensor([True, True, True, True])

[**现在计算`x`的另一个函数。**]


In [ ]:
# 在默认情况下，PyTorch会累积梯度，我们需要清除之前的值
x.grad.zero_()
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

## 非标量变量的反向传播

当`y`不是标量时，向量`y`关于向量`x`的导数的最自然解释是一个矩阵。
对于高阶和高维的`y`和`x`，求导的结果可以是一个高阶张量。

然而，虽然这些更奇特的对象确实出现在高级机器学习中（包括[**深度学习中**]），
但当调用向量的反向计算时，我们通常会试图计算一批训练样本中每个组成部分的损失函数的导数。
这里(**我们的目的不是计算微分矩阵，而是单独计算批量中每个样本的偏导数之和。**)


In [ ]:
# 对非标量调用backward需要传入一个gradient参数，该参数指定微分函数关于self的梯度。
# 本例只想求偏导数的和，所以传递一个1的梯度是合适的
x.grad.zero_()
y = x * x
# 等价于y.backward(torch.ones(len(x)))
y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

## 一、自动微分的基本思想

自动微分不是符号微分（Symbolic Diff），也不是数值微分（Numerical Diff），而是**利用链式法则，在计算函数值的同时或之后，精确计算导数**。

它有两种主要模式：
- **正向模式（Forward Mode）**
- **反向模式（Reverse Mode）**

> ⚠️ 注意：**“正向传播” ≠ “正向模式自动微分”**  
> 在深度学习中，“前向传播”指计算 loss，而“反向传播”指用**反向模式 AD** 计算梯度。
- **Loss（损失）**：标量函数 $ L(\theta) $，衡量模型预测与真实值的差距。
- **Gradient（梯度）**：$ \nabla_\theta L = \left( \frac{\partial L}

---

## 二、正向模式自动微分（Forward Mode AD）

### ✅ 原理
- 沿着计算图**从输入到输出**的方向，同时计算函数值和**对某一输入变量的偏导数**
- 每个中间变量 $ v_i $ 附带一个“对种子变量 $ x_j $ 的导数” $ \dot{v}_i = \frac{\partial v_i}{\partial x_j} $
- 使用链式法则逐层传播

### 🔑 特点
| 特性 | 说明 |
|------|------|
| **计算方向** | 输入 → 输出 |
| **一次计算** | 得到 **一个输入变量** 对所有输出的导数（Jacobian 的一列） |
| **时间复杂度** | $ O(n) \times \text{cost}(f) $，其中 $ n $ 是输入维度 |
| **空间复杂度** | $ O(1) $（无需存储中间结果） |
| **适用场景** | 输入少、输出多（如 $ f: \mathbb{R} \to \mathbb{R}^m $） |

### 📌 在深度学习中的作用
- **几乎不用**于训练：因为神经网络通常有 $ P \sim 10^6 \sim 10^9 $ 个参数，计算全部梯度需运行 $ P $ 次，**效率极低**。
- 用于**灵敏度分析（某个输入扰动对输出的影响）**、**物理模拟**、**Hessian-vector product（用于二阶优化）** 等

---

## 三、反向模式自动微分（Reverse Mode AD）

### ✅ 原理
- 分两阶段：
  1. **前向阶段**：计算函数值，**记录计算图（tape）**
  2. **反向阶段**：从输出开始，反向应用链式法则，计算**所有输入对某一个输出的梯度**
- 引入伴随变量（adjoint）$ \bar{v}_i = \frac{\partial L}{\partial v_i} $

### 🔑 特点
| 特性 | 说明 |
|------|------|
| **计算方向** | 输出 ← 输入（反向） |
| **一次计算** | 得到 **所有输入变量** 对一个输出（如 loss）的梯度（Jacobian 的一行） |
| **时间复杂度** | $ O(1) \times \text{cost}(f) $（常数倍） |
| **空间复杂度** | $ O(\text{depth}) $（需存储前向中间结果） |
| **适用场景** | 输入多、输出少（如 $ f: \mathbb{R}^n \to \mathbb{R} $）→ **正是深度学习！** |

### 📌 在深度学习中的作用
- **核心机制**：**反向传播（Backpropagation）就是反向模式 AD 在神经网络中的具体实现**
- 高效计算百万/十亿参数的梯度
- 支撑了 SGD、Adam、L-BFGS 等所有基于梯度的优化器

---

## 四、对 Gradient 和 Loss 的影响

|          | 正向模式 AD | 反向模式 AD |
|----------|-------------|-------------|
| **Loss 计算** | 同时计算 loss 和某个参数的 ∂loss/∂θᵢ | 先计算 loss，再反向计算所有 ∂loss/∂θ |
| **Gradient 计算效率** | 若有 $ P $ 个参数，需 $ P $ 次正向 → **极慢** | 1 次反向 → **高效** |
| **内存占用** | 低 | 高（需存前向激活值） |
| **实际使用** | ❌ 不用于训练 | ✅ 所有深度学习框架默认 |

> 💡 **为什么深度学习用反向模式？**  
> 因为 loss 是标量（输出=1），参数极多（输入=10⁶~10⁹），反向模式只需 **1 次反向 pass** 即可得到全部梯度！

---

## 五、非平凡样例

### ✅ 样例 1：正向模式 AD（非平凡）

**任务**：计算函数  
$$
f(x, y) = \sin(x \cdot y) + \exp(x + y)
$$  
在点 $ (x, y) = (1, 2) $ 处对 $ x $ 的偏导数 $ \frac{\partial f}{\partial x} $

#### 正向模式计算过程（带导数传播）：

| 变量 | 值 $ v $ | 导数 $ \dot{v} = \partial v / \partial x $ |
|------|-----------|--------------------------------------------|
| $ v_1 = x $ | 1 | 1 |
| $ v_2 = y $ | 2 | 0 |
| $ v_3 = v_1 \cdot v_2 = 2 $ | 2 | $ \dot{v}_1 v_2 + v_1 \dot{v}_2 = 1 \cdot 2 + 1 \cdot 0 = 2 $ |
| $ v_4 = \sin(v_3) $ | $ \sin(2) \approx 0.909 $ | $ \cos(v_3) \cdot \dot{v}_3 \approx \cos(2) \cdot 2 \approx -0.832 $ |
| $ v_5 = v_1 + v_2 = 3 $ | 3 | $ \dot{v}_1 + \dot{v}_2 = 1 + 0 = 1 $ |
| $ v_6 = \exp(v_5) $ | $ e^3 \approx 20.086 $ | $ \exp(v_5) \cdot \dot{v}_5 \approx 20.086 \cdot 1 = 20.086 $ |
| $ f = v_4 + v_6 $ | ≈ 20.995 | $ \dot{v}_4 + \dot{v}_6 \approx -0.832 + 20.086 = 19.254 $ |

✅ 结果：$ \frac{\partial f}{\partial x} \approx 19.254 $

> 🔍 这就是正向模式：**一次计算得到一个偏导**。若还要 $ \partial f/\partial y $，需重新设种子 $ \dot{y}=1, \dot{x}=0 $ 再跑一次。

---

### ✅ 样例 2：反向模式 AD（非平凡，深度学习场景）

**任务**：一个简单的 RNN 单元（非平凡，因含循环依赖）

$$
\begin{aligned}
h_t &= \tanh(W_{hh} h_{t-1} + W_{xh} x_t + b_h) \\
L &= \frac{1}{2} \|h_T - y\|^2 \quad \text{(MSE loss at final step)}
\end{aligned}
$$

假设 $ T=3 $，我们要计算 $ \frac{\partial L}{\partial W_{hh}} $

#### 反向模式过程：

1. **前向计算**（记录 $ h_0, h_1, h_2, h_3 $）：
   - $ h_1 = \tanh(W_{hh} h_0 + W_{xh} x_1 + b_h) $
   - $ h_2 = \tanh(W_{hh} h_1 + W_{xh} x_2 + b_h) $
   - $ h_3 = \tanh(W_{hh} h_2 + W_{xh} x_3 + b_h) $
   - $ L = \frac{1}{2} \|h_3 - y\|^2 $

2. **反向传播**（从 $ L $ 开始）：
   - $ \bar{h}_3 = \frac{\partial L}{\partial h_3} = h_3 - y $
   - $ \bar{a}_3 = \bar{h}_3 \odot (1 - h_3^2) $ （tanh 导数）
   - $ \bar{W}_{hh}^{(3)} = \bar{a}_3 h_2^\top $
   - $ \bar{h}_2 = W_{hh}^\top \bar{a}_3 $
   - $ \bar{a}_2 = \bar{h}_2 \odot (1 - h_2^2) $
   - $ \bar{W}_{hh}^{(2)} = \bar{a}_2 h_1^\top $
   - $ \bar{h}_1 = W_{hh}^\top \bar{a}_2 $
   - $ \bar{a}_1 = \bar{h}_1 \odot (1 - h_1^2) $
   - $ \bar{W}_{hh}^{(1)} = \bar{a}_1 h_0^\top $

3. **总梯度**：
   $$
   \frac{\partial L}{\partial W_{hh}} = \bar{W}_{hh}^{(1)} + \bar{W}_{hh}^{(2)} + \bar{W}_{hh}^{(3)}
   $$

✅ 这就是**BPTT（Backpropagation Through Time）**，是反向模式 AD 在 RNN 中的应用。

> 🔍 关键：**一次反向 pass** 就得到了 $ W_{hh} $ 的完整梯度，尽管它影响了 $ h_1, h_2, h_3 $ 三个时刻！

---

## 六、总结对比表

| 维度 | 正向模式 AD | 反向模式 AD |
|------|------------|------------|
| **计算方向** | 输入 → 输出 | 输出 ← 输入 |
| **一次计算得** | 一个输入的梯度 | 所有输入的梯度 |
| **时间复杂度** | $ O(n) \cdot \text{cost}(f) $ | $ O(1) \cdot \text{cost}(f) $ |
| **空间复杂度** | $ O(1) $ | $ O(T) $（需存前向值） |
| **深度学习适用性** | ❌ 极低（参数太多） | ✅ **标准方法** |
| **典型应用** | 灵敏度分析、ODE 求解 | 神经网络训练、损失优化 |

---

## 💡 最终结论

- **正向模式 AD**：适合“少量输入，大量输出”的场景，**不用于深度学习训练**。
- **反向模式 AD**：完美匹配“海量参数，单个 loss”的深度学习范式，是 `backward()` 的数学基础。
- **自动微分不是近似**：它通过链式法则**精确计算梯度**（忽略浮点误差）。
- **反向模式的代价是内存**：这也是为什么大模型需要梯度检查点（Gradient Checkpointing）来 trade-off 时间与空间。


## 分离计算

有时，我们希望[**将某些计算移动到记录的计算图之外**]。
例如，假设`y`是作为`x`的函数计算的，而`z`则是作为`y`和`x`的函数计算的。
想象一下，我们想计算`z`关于`x`的梯度，但由于某种原因，希望将`y`视为一个常数，
并且只考虑到`x`在`y`被计算后发挥的作用。

这里可以分离`y`来返回一个新变量`u`，该变量与`y`具有相同的值，
但丢弃计算图中如何计算`y`的任何信息。
换句话说，梯度不会向后流经`u`到`x`。
因此，下面的反向传播函数计算`z=u*x`关于`x`的偏导数，同时将`u`作为常数处理，
而不是`z=x*x*x`关于`x`的偏导数。


In [ ]:
import torch

x = torch.tensor([1.0, 2.0], requires_grad=True)
x.grad.zero_()
y = x * x
u = y.detach() # u 是 y 的“副本”，但不参与梯度计算
z = u * x # z = u * x = (x²)_detached * x

print(f'z = {z},z.sum() = {z.sum()}')
z.sum().backward()
x.grad == u

NameError: name 'torch' is not defined

这是一个**极其重要且常见**的困惑！你触及了 PyTorch 自动微分（autograd）中 **梯度累积（gradient accumulation）** 和 **计算目标（what we differentiate）** 的核心机制。我们来彻底澄清。

---

## ✅ 一、根本原则：`.backward()` 总是计算 **loss 对所有 `requires_grad=True` 叶子张量的梯度**

### 关键点：
- **`z.sum().backward()` 并不是“求 x 的导数”**，而是：
  > **以 `z.sum()` 为标量损失（scalar loss），计算它对所有参与计算的、`requires_grad=True` 的叶子变量（如 `x`）的梯度，并累加到它们的 `.grad` 属性中。**

- **`z.grad` 通常不存在**（除非 `z.requires_grad=True` 且被更高层使用）

---

## 🔍 二、为什么是 `x.grad` 而不是 `z.grad`？

### 回顾你的代码：
```python
x = torch.tensor([2.0], requires_grad=True)
y = x * x          # y.requires_grad = True
u = y.detach()     # u.requires_grad = False
z = u * x          # z.requires_grad = True（因为 x 有梯度）
z.sum().backward()
```

### 分析：
| 张量 | `requires_grad` | 是否是叶子（leaf） | 是否有 `.grad` |
|------|------------------|-------------------|---------------|
| `x` | `True` | ✅ 是（用户创建） | ✅ 有（用于存储 ∂L/∂x）|
| `y` | `True` | ❌ 否（由 x 计算得来） | ❌ 无（中间变量）|
| `u` | `False` | ✅ 是 | ❌ 无 |
| `z` | `True` | ❌ 否 | ❌ **通常无** |

> 📌 **只有 `requires_grad=True` 的叶子张量才有 `.grad` 属性！**

- `z` 是中间结果（非叶子），所以 **`z.grad` 默认不保存**
- 我们关心的是**参数 `x` 的梯度**，所以梯度存到 `x.grad`

> 💡 如果你真的想看 `z` 对自己的梯度（总是 1），可以：
> ```python
> z.retain_grad()  # 显式要求保存中间变量梯度
> z.sum().backward()
> print(z.grad)    # tensor([1., 1., ...])
> ```

---

## 🔁 三、梯度累积：多次 `.backward()` 会累加！

这是你第二个问题的核心：

> “如果我先对 `y` 调用 backward，再对 `z` 调用 backward，`x.grad` 是谁的？”

### ✅ 答案：**`x.grad` 是两者的梯度之和！**

PyTorch **默认累加梯度**（这是为了支持梯度累积训练大 batch）。

### 🌰 举例：
```python
x = torch.tensor([2.0], requires_grad=True)

# 第一次：以 y 为 loss
y = x * x
y.backward()        # ∂y/∂x = 2x = 4 → x.grad = [4.]

# 第二次：以 z 为 loss
z = x * x * x       # z = x³
z.backward()        # ∂z/∂x = 3x² = 12 → x.grad += 12 → x.grad = [16.]

print(x.grad)       # tensor([16.]) = 4 + 12
```

### ❗ 所以：**`x.grad` 始终是所有已调用 `.backward()` 的 loss 对 `x` 的梯度之和**

---

## 🛑 四、如何避免混淆？—— 标准做法

在深度学习训练循环中，**每次迭代前必须清零梯度**：

```python
for data, target in dataloader:
    optimizer.zero_grad()      # ← 等价于 x.grad = None 或 x.grad.zero_()
    
    output = model(data)
    loss1 = criterion1(output, target)
    loss1.backward()           # 计算 ∂loss1/∂params
    
    # 可能还有其他 loss
    loss2 = some_other_loss(output)
    loss2.backward()           # 梯度累加：∂(loss1+loss2)/∂params
    
    optimizer.step()           # 用总梯度更新参数
```

> ✅ **`optimizer.zero_grad()` 的作用就是防止上一轮的梯度污染本轮！**

---

## 📊 五、总结：谁的梯度存在哪里？

| 问题 | 答案 |
|------|------|
| **`z.sum().backward()` 输出是 `z.grad` 吗？** | ❌ 不是！`z` 是中间变量，无 `.grad`（除非 `retain_grad()`） |
| **梯度存在哪里？** | ✅ 存在**所有 `requires_grad=True` 的叶子张量**的 `.grad` 中（如模型参数 `x`） |
| **多次 `backward()` 会覆盖还是累加？** | ✅ **累加！**（`x.grad += new_gradient`） |
| **如何知道 `x.grad` 对应哪个 loss？** | 🔑 **它对应所有已调用 `backward()` 的 loss 的梯度之和** |
| **如何只保留当前 loss 的梯度？** | ✅ 在 `backward()` 前调用 `x.grad = None` 或 `optimizer.zero_grad()` |

---

## 💡 终极理解

- **`.backward()` 的输入是一个标量（loss）**
- **它的输出（梯度）自动分发到所有可学习参数（叶子张量）**
- **`.grad` 是参数的属性，不是 loss 的属性**
- **梯度默认累加，这是 feature 不是 bug！**

由于记录了`y`的计算结果，我们可以随后在`y`上调用反向传播，
得到`y=x*x`关于的`x`的导数，即`2*x`。


In [9]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

## Python控制流的梯度计算

使用自动微分的一个好处是：
[**即使构建函数的计算图需要通过Python控制流（例如，条件、循环或任意函数调用），我们仍然可以计算得到的变量的梯度**]。
在下面的代码中，`while`循环的迭代次数和`if`语句的结果都取决于输入`a`的值。


In [10]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

让我们计算梯度。


In [11]:
a = torch.randn(size=(), requires_grad=True)
d = f(a)
d.backward()

我们现在可以分析上面定义的`f`函数。
请注意，它在其输入`a`中是分段线性的。
换言之，对于任何`a`，存在某个常量标量`k`，使得`f(a)=k*a`，其中`k`的值取决于输入`a`，因此可以用`d/a`验证梯度是否正确。


In [12]:
a.grad == d / a

tensor(True)

## 小结

* 深度学习框架可以自动计算导数：我们首先将梯度附加到想要对其计算偏导数的变量上，然后记录目标值的计算，执行它的反向传播函数，并访问得到的梯度。

## 练习

1. 为什么计算二阶导数比一阶导数的开销要更大？
1. 在运行反向传播函数之后，立即再次运行它，看看会发生什么。
1. 在控制流的例子中，我们计算`d`关于`a`的导数，如果将变量`a`更改为随机向量或矩阵，会发生什么？
1. 重新设计一个求控制流梯度的例子，运行并分析结果。
1. 使$f(x)=\sin(x)$，绘制$f(x)$和$\frac{df(x)}{dx}$的图像，其中后者不使用$f'(x)=\cos(x)$。


[Discussions](https://discuss.d2l.ai/t/1759)
